## Sector Público: Scraping de URLs

**Fecha de ejecución:** 22/07/2025  
**Empresas objetivo:** Agencia Tributaria (AEAT), Seguridad Social, DGT, SEPE, INEM, Suma Gestión Tributaria, Catastro

 | Empresa                    | URLs únicas | Fecha      | Estado / Observaciones           |
|----------------------------|------------|------------|----------------------------------|
| DGT                        |    287     | 22/07/2025 | OK                               |
| SEPE                       |    105     | 22/07/2025 | OK                               |
| Agencia Tributaria (AEAT)  |    101     | 22/07/2025 | OK                               |
| Seguridad Social           |     74     | 22/07/2025 | OK                               |
| Suma Gestión Tributaria    |     59     | 22/07/2025 | OK                               |
| Catastro                   |     21     | 22/07/2025 | OK                               |
| INEM                       |      1     | 22/07/2025 | OK – Muy pocos enlaces           |


In [1]:
# Importamos las librerías necesarias para el scraping y el análisis de datos
import requests                             # Para hacer peticiones HTTP a las webs
from bs4 import BeautifulSoup               # Para parsear el HTML de la página web
from urllib.parse import urlparse, urljoin  # Para trabajar con URLs y dominios
import pandas as pd                         # Para almacenar y exportar los datos en DataFrame
import time                                 # Para pausar entre peticiones y evitar bloqueos


In [2]:
def obtener_urls(base_url, delay=1):
    """
    Descarga la página principal de una web y extrae todas las URLs internas.
    Solo añade URLs que pertenezcan al dominio principal o subdominios de la empresa.
    - base_url: URL principal de la empresa
    - delay: segundos de espera tras cada request para evitar bloqueos
    """
    try:
        # Definimos un User-Agent "realista" para que el servidor no bloquee la petición por bot
        headers = {
            "User-Agent": (
                "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                "AppleWebKit/537.36 (KHTML, like Gecko) "
                "Chrome/91.0.4472.124 Safari/537.36"
            )
        }
        # Hacemos la petición HTTP a la web (con timeout de 10 segundos)
        response = requests.get(base_url, timeout=10, headers=headers)

        # Parseamos el HTML recibido con BeautifulSoup
        soup = BeautifulSoup(response.text, 'html.parser')

        # Extraemos el dominio principal (ej: 'amazon.es' de 'www.amazon.es')
        dominio_empresa = urlparse(base_url).netloc
        if dominio_empresa.startswith('www.'):
            dominio_base = dominio_empresa[4:]
        else:
            dominio_base = dominio_empresa

        # Creamos un set para guardar solo URLs únicas
        urls = set()

        # Buscamos todos los enlaces <a> que tengan atributo href
        for link in soup.find_all('a', href=True):
            href = link['href']                  # Obtenemos el href
            href = urljoin(base_url, href)       # Convertimos a URL absoluta
            dominio_href = urlparse(href).netloc # Dominio del enlace

            # Eliminamos 'www.' para comparar dominios correctamente
            if dominio_href.startswith('www.'):
                dominio_href_base = dominio_href[4:]
            else:
                dominio_href_base = dominio_href

            # Añadimos solo si es del dominio principal o subdominio
            if dominio_href_base == dominio_base or dominio_href_base.endswith('.' + dominio_base):
                urls.add(href)

        # Esperamos el tiempo indicado (para no ser bloqueados por el servidor)
        time.sleep(delay)

        # Devolvemos la lista de URLs y un estado "OK"
        return list(urls), "OK"

    except Exception as e:
        # Si hay error, lo devolvemos como estado y lista vacía de URLs
        return [], str(e)


In [6]:
empresas_sector_publico = {
    "Agencia Tributaria (AEAT)": "https://www.agenciatributaria.es",
    "Seguridad Social": "https://www.seg-social.es",
    "DGT": "https://www.dgt.es",
    "SEPE": "https://www.sepe.es",
    "INEM": "https://www.inem.es",
    "Suma Gestión Tributaria": "https://www.suma.es",
    "Catastro": "https://www.sedecatastro.gob.es"
}


In [7]:
# Lista donde almacenaremos los resultados de todas las empresas
resultados = []

# Guardamos la fecha actual
fecha = pd.Timestamp.today().strftime("%d/%m/%Y")

# Iteramos sobre cada empresa y su URL
for nombre, url_base in empresas_sector_publico.items():
    print(f'Scraping empresa: {nombre}')                # Log para saber el progreso
    urls, estado = obtener_urls(url_base)               # Llamamos a la función de scraping
    if urls:
        # Si hay URLs, añadimos cada una como fila en los resultados
        for url in urls:
            resultados.append({
                'empresa': nombre,
                'url': url,
                'fecha': fecha,
                'estado': estado
            })
    else:
        # Si falla o no hay URLs, registramos igualmente el intento
        resultados.append({
            'empresa': nombre,
            'url': '',
            'fecha': fecha,
            'estado': estado
        })

# Convertimos la lista de resultados en un DataFrame de pandas
df_scrap = pd.DataFrame(resultados)

# Guardamos el DataFrame como CSV en la carpeta correspondiente (cambia el nombre por sector)
df_scrap.to_csv('../data/raw/publico_legitimas_crudo.csv', index=False)

# Mostramos por pantalla las primeras filas del DataFrame para revisión rápida
print(df_scrap.head())


Scraping empresa: Agencia Tributaria (AEAT)
Scraping empresa: Seguridad Social
Scraping empresa: DGT
Scraping empresa: SEPE
Scraping empresa: INEM
Scraping empresa: Suma Gestión Tributaria
Scraping empresa: Catastro
                     empresa  \
0  Agencia Tributaria (AEAT)   
1  Agencia Tributaria (AEAT)   
2  Agencia Tributaria (AEAT)   
3  Agencia Tributaria (AEAT)   
4  Agencia Tributaria (AEAT)   

                                                 url       fecha estado  
0  https://www.agenciatributaria.es/Sede/informac...  22/07/2025     OK  
1  https://www.agenciatributaria.es/Sede/identifi...  22/07/2025     OK  
2  https://www.agenciatributaria.es/Sede/estadist...  22/07/2025     OK  
3  https://www.agenciatributaria.es/Sede/ayuda/di...  22/07/2025     OK  
4  https://www.agenciatributaria.es/Sede/certific...  22/07/2025     OK  


In [8]:
print(df_scrap['empresa'].value_counts())




empresa
DGT                          287
SEPE                         105
Agencia Tributaria (AEAT)    101
Seguridad Social              74
Suma Gestión Tributaria       59
Catastro                      21
INEM                           1
Name: count, dtype: int64


### Resumen y análisis del scraping (Sector Público)

- **Empresas scrapeadas:** 7
- **Total URLs obtenidas:** 648
- **Patrones detectados:**
    - DGT y SEPE destacan con alto número de URLs públicas.
    - INEM y Catastro apenas muestran enlaces (probable home muy básica o minimalista).
- **Incidencias:**
    - Ningún bloqueo aparente ni errores críticos; la mayoría de portales permiten scraping simple.
- **Siguientes pasos:**
    - Si interesa, scrapear secciones internas en empresas con pocos enlaces (INEM, Catastro).
    - Continuar con el siguiente sector.
